In [1]:
import sys
sys.path.append('../Modules')
from read_prepare_input_clustering import *
from preprocess_series import *
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance
from arch import arch_model
import time

# Read input data

df_all, df_all_lookup, _, _, _ =\
    import_data_clustering_issuer_rf(issuer_curves_file_name='Basedati3_spread.csv',
                                    fx_rf_file_name='../Data/Observations - UniverseFX.csv',
                                    local_rf_file_name='../Data/Observations - UniverseLOCAL.csv',
                                    sep_issuer_curves_file=';')

df_complete = remove_series_mostly_na(df_all_lookup, 0.75)# df_complete = df_all_lookup.copy()
returnDf = df_complete.pct_change()

Maximum Timestamp of available data point 2016-11-30 00:00:00
Minimum Timestamp of not null data point cross time series 2022-02-04 00:00:00


In [2]:
def archFit(retDf,lagP,lagO,lagQ,p,scaling,resDist):
    retDf = retDf[~retDf.isnull()]
    if pd.notnull(scaling):
        model = arch_model(retDf.astype('float')*scaling, 
                    p=lagP, o=lagO, q=lagQ,
                    power=p,dist=resDist)
    else:
         model = arch_model(retDf.astype('float'), 
                    p=lagP, o=lagO, q=lagQ,rescale=True,
                    power=p,dist=resDist)
    fittedModel = model.fit(disp=False)
    std_resid = fittedModel.resid / fittedModel.conditional_volatility
    outputList =  [fittedModel.conditional_volatility,std_resid]
    return(outputList)

def applyAarch(retDf,lagP,lagO,lagQ,p,scaling,resDist):
    factorDataDict = {}
    for iCol in retDf:
        print(iCol)
        factorRet = retDf[iCol]
        #codVola = None
        #stdRet = None
        if not (np.issubdtype(factorRet.dtype, np.datetime64)):
            factorRet = factorRet.astype("float")
            processedFactor= archFit(factorRet,lagP,lagO,lagQ,p,scaling,resDist)
            codVola = processedFactor[0]
            stdRet = processedFactor[1]
        else:
            codVola = factorRet
            stdRet = factorRet
        factorDataDict[iCol]={"conditionalVola":codVola,"stdReturn":stdRet}
    return(factorDataDict)

def distanceCalculationIssuer(outputApplyArch):
    issuers = list(outputApplyArch.keys())
    distanceStd = []
    distanceCVola = []
    for iFact in issuers:
        tmpStd = []
        tmpCVola = []
        for jFact in issuers:
            tmpStd.append(euclideanDistance(outputApplyArch[iFact]["stdReturn"],outputApplyArch[jFact]["stdReturn"]))
            tmpCVola.append(euclideanDistance(outputApplyArch[iFact]["conditionalVola"],outputApplyArch[jFact]["conditionalVola"]))
        distanceStd.append(tmpStd)
        distanceCVola.append(tmpCVola)
    distMatDict = {"dCVola":distanceCVola,"dStdRet":distanceStd,"factorNames":issuers}
    return(distMatDict)

In [3]:
decomposedFactors = applyAarch(returnDf,lagP=1,lagO=0,lagQ=3,p=2,scaling=None,resDist="StudentsT")

AALLN-19392-EUR-SNR-BND
AARB-21310-EUR-SNR-BND
ABBLTD-23-USD-SNR-BND
ABT-2254033-USD-SNR-BND
ABT-23457-USD-SNR-BND
ACAFP-18185-EUR-SNR-BND
ACCOR-65-EUR-SNR-BND


/anaconda/envs/work_env/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


ACEIM-24415-EUR-SNR-BND
ACESM-77-EUR-SNR-BND
ACE_INAHLDGS-120755-USD-SNR-BND
ADGLHO-3946887-EUR-SNR-BND
ADHABI-89935-USD-SNR-BND
ADIFAL-3364579-EUR-SNR-BND
ADLERR-100334-EUR-SNR-BND
ADO-17026-EUR-SNR-BND
ADPFP-18004-EUR-SNR-BND
ADSGR-22940-EUR-SNR-BND
AEGON-14749-EUR-SNR-BND
AEMSPA-23344-EUR-SNR-BND
AERRIA-16115-EUR-SNR-BND
AGFRNC-14970-EUR-SNR-BND
AHOLD-20969-EUR-SNR-BND
AIFP-122795-EUR-SNR-BND
AIFP_SA-18631-EUR-SNR-BND
AIG-14406-USD-SNR-BND
AKEFP-109891-EUR-SNR-BND
AKFAST-1191331-EUR-SNR-BND
AKZO-14768-EUR-SNR-BND
ALATPF-3576896-EUR-SNR-BND
ALZSE-17338-EUR-SNR-BND
AMB_PROPERTY-119304-USD-SNR-BND
AMGN-25256-USD-SNR-BND


/anaconda/envs/work_env/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


AMROBK-159062-EUR-SNR-BND
AMT-456-USD-SNR-BND
AMXLMM-18009-USD-SNR-BND
ANZ-14807-USD-SNR-BND
AOC-14812-USD-SNR-BND
AOXGR-85498-EUR-SNR-BND
APAAU-48738-USD-SNR-BND
APD-528-USD-SNR-BND
APHP-534-EUR-SNR-BND
APLINC-11-USD-SNR-BND
ASFFP-89152-EUR-SNR-BND
ASML-682-EUR-SNR-BND
ASSGEN-22965-EUR-SNR-BND
AT&T_INC-15691-USD-SNR-BND
ATCOA-711-EUR-SNR-BND
ATDBCN-18244-EUR-SNR-BND
ATH-21707-USD-SNR-BND
ATRSAV-22701-EUR-SNR-BND
AUCHAN-130697-EUR-SNR-BND
AUTSTR-24302-EUR-SNR-BND
AVLN-14252-EUR-SNR-BND
AXASA-14081-EUR-SCR-BND
AXASA-14081-EUR-SNR-BND
AZN-19637-USD-SNR-BND
BA-837-USD-SNR-BND
BABA-156978-USD-SNR-BND
BABLN-99824-EUR-SNR-BND
BAC-14849-USD-SNR-BND
BACR-16072-EUR-SNR-BND
BACRED-851-EUR-SNR-BND
BADGR_ENBWIN-122736-EUR-SNR-BND
BANSAB-947-EUR-SNR-BND
BASF-16633-EUR-SNR-BND
BATSLN-14196-EUR-SNR-BND
BAX-14871-USD-SNR-BND
BAYLAN-1003-EUR-SNR-BND
BBVSM-14374-EUR-SNR-BND
BDX-1073-USD-SNR-BND
BELGBB-14728-EUR-SNR-BND
BERN-17295-CHF-SNR-BND
BERTEL-1133-EUR-SNR-BND
BFB-14888-USD-SNR-BND
BFCM-1156-EUR-SN

...

TELNOR-12229-EUR-SNR-BND
TEMASE-26660-EUR-SNR-BND
TENN-156758-EUR-SNR-BND
TEREOS-2953230-EUR-SNR-BND
TEVA-12263-USD-SNR-BND
THURBK-12317-CHF-SNR-BND
TIF-12331-USD-SNR-BND
TIPIM-109883-EUR-SNR-BND
TITIM-115069-EUR-SNR-BND
TITKGA-1314980-EUR-SNR-BND
TKA-22572-EUR-SNR-BND
TKAGR-12353-EUR-SNR-BND
TLIASS-14059-EUR-SNR-BND
TMCC-150381-USD-SNR-BND
TMO-14364-USD-SNR-BND
TOTAL-24978-EUR-SNR-BND
TOTAL-503153-EUR-SNR-BND
TRANLN-24755-GBP-SNR-BND
TRNIM-24641-EUR-SNR-BND
TRP-122537-USD-SNR-BND
TVA-17037-USD-SNR-BND
TYC-15830-EUR-SNR-BND
UBS-14046-EUR-SNR-BND
UBS-3534636-USD-SNR-BND
UCIM-14227-EUR-SNR-BND
ULVR-14075-EUR-SNR-BND
UNEDIC-12832-EUR-SNR-BND
UNFSM_FIN-122660-EUR-SNR-BND
UNH-24844-USD-SNR-BND
UNIPOL-115310-EUR-SNR-BND
UNIROD-12827-EUR-SNR-BND
UNP-17703-USD-SNR-BND
UPS-14405-USD-SNR-BND
URENCO-12918-EUR-SNR-BND
UTX-15861-USD-SNR-BND
VATFAL-15862-EUR-SNR-BND
VED-24770-USD-SNR-BND
VERBND-14445-EUR-SNR-BND
VFC-13404-USD-SNR-BND
VGASDE-2667706-EUR-SNR-BND
VIA-30299-USD-SNR-BND
VIEFP-20150-EUR-S

/anaconda/envs/work_env/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/anaconda/envs/work_env/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


VMED_FIN-124344-EUR-SNR-BND
VNESH-114168-USD-SNR-BND
VNU-24833-USD-SNR-BND
VOD-14085-EUR-SNR-BND
VOVCAB-3923908-EUR-SNR-BND
VTR-24720-USD-SNR-BND
VW-24188-EUR-SNR-BND
VZ-14136-USD-SNR-BND
WAG-13573-USD-SNR-BND
WDC-13616-USD-SNR-BND
WESAU-20385-EUR-SNR-BND


/anaconda/envs/work_env/lib/python3.9/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


WHR-13685-USD-SNR-BND
WKBSW-12963-CHF-SNR-BND
WOLKLU-13756-EUR-SNR-BND
WPC-45167-USD-SNR-BND
WPDHLD-124277-GBP-SNR-BND
WPPLN-20216-EUR-SNR-BND
WRTAU-2967093-EUR-SNR-BND
WSTP-15920-USD-SNR-BND
WU-31868-USD-SNR-BND
XOM-14203-USD-SNR-BND
XTALN-22833-USD-SNR-BND
YANTZE-13901-USD-SNR-BND
ZFFNGR-3673421-EUR-SNR-BND
ZKB-21978-CHF-SNR-BND
ZUGKAN-14033-CHF-SNR-BND
ZURNVX-121028-CHF-SNR-BND
ZURNVX-22877-EUR-SNR-BND


In [4]:
cond_vola_df = pd.DataFrame()

for issuer in decomposedFactors:
    df_curr_issuer = pd.DataFrame(decomposedFactors[issuer]['conditionalVola']).rename(columns={'cond_vol': issuer})
    cond_vola_df = pd.concat([cond_vola_df, df_curr_issuer], axis=1)
    
cond_vola_df.to_csv('../Data/conditional_volatility_issuers_BaseDati3.csv', sep=';')